# Lab 2 - Comprehensive Explanations of Hospital LOS Models

In [ ]:
# This code is based on Reta Giblin work for Project 1
# This lab includes:
#     Data explanation using SweetViz (this is at the end due to library compatibility issues)
#     Global explanation using SHAP summary, PDP, ALE
#     Local explantion using SHAP, LIME, counterfactuals

# Attempts are also made to use the original data for explanation (rather than normalized/encoded data)
# However, some explanation packages make this difficult (e.g. interpretML)

# Reta GiblinM
# DSBA-6010 Project 1
# 
# Fall 2023
# Data Preprocessing & Supervised Learning Models

In [ ]:
# Uncomment and run the following the first time and then put comment back on
# Python 3.9 is recommended due to compatibility issues of various packages

#%pip install interpret alibi shap dice-ml statsmodels seaborn ipywidgets sweetviz

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_score, roc_auc_score, mean_squared_error, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer

pd.set_option('display.max_columns', 200)

In [ ]:
# Read dataset

In [ ]:
#dtype_dict = {'dialysisrenalendstage': bool, 'asthma': bool, 'irondef': bool, 'pneum': bool, 'substancedependence': bool, 
#              'psychologicaldisordermajor': bool, 'depress': bool,  'psychother': bool,  'fibrosisandother': bool, 
#              'malnutrition': bool, 'hemo': bool,}
dtype_dict = {'dialysisrenalendstage': int, 'asthma': int, 'irondef': int, 'pneum': int, 'substancedependence': int, 
              'psychologicaldisordermajor': int, 'depress': int,  'psychother': int,  'fibrosisandother': int, 
              'malnutrition': int, 'hemo': int,}

# set up to read date fields as dates
date_columns = ['vdate', 'discharged']
los_data = pd.read_csv("LengthOfStay.csv", dtype = dtype_dict, parse_dates=date_columns)

In [ ]:
# DATA PREPROCESSING/TRANSFORMATION
los_data.dtypes

In [ ]:
# Create a boolean column based on the 'gender' column
los_data['is_female'] = los_data['gender'] == 'F'

# Convert the boolean column to boolean data type
#los_data['is_female'] = los_data['is_female'].astype(bool)
los_data['is_female'] = los_data['is_female'].astype(int)

# Transform lengthofstay to binary field
# Create a Boolean field name longlengthstay
# If lengthofstay Less that 7 days set longlengthofstay to 0 if >= 7 days set longlengthofstay to 1

los_data['longlengthofstay'] = los_data['lengthofstay'] >= 7


In [ ]:
# DATA ENCODING


In [ ]:
# Create dummy variables for the facid column

los_data = pd.get_dummies(los_data, columns=['facid'], prefix='facid')

In [ ]:
# Change rcount categorical value to an integer
# Replace '5+' with '5' in the facid column
los_data['rcount'] = los_data['rcount'].astype(str) 
los_data['rcount'] = los_data['rcount'].str.replace('5+', '5', regex=False)

# Change type of facid to integer
los_data['rcount'] = los_data['rcount'].astype(int)

In [ ]:
# Drop unneeded columns
column_to_drop = ['gender', 'lengthofstay','eid']
los_data = los_data.drop(column_to_drop, axis=1)

In [ ]:
# REMOVING OUTLIERS

In [ ]:
# Define a function to remove outliers based on Z-score
def remove_outliers(df, columns, threshold=3):
    z_scores = np.abs((df[columns] - df[columns].mean()) / df[columns].std())
    df_no_outliers = df[(z_scores < threshold).all(axis=1)]
    return df_no_outliers

In [ ]:
# Remove outliers for numerical data
numeric_columns = ['hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi', 'pulse','respiration']

# Remove rows with outliers (Z-score threshold = 3)
los_data = remove_outliers(los_data, numeric_columns)

In [ ]:
# Feature Engineer vdate date

# Extract vdate individual elements
los_data['vdate_year'] = los_data['vdate'].dt.year
los_data['vdate_month'] = los_data['vdate'].dt.month
los_data['vdate_day'] = los_data['vdate'].dt.day
los_data['vdate_day_of_week'] = los_data['vdate'].dt.dayofweek

# Convert vdate elements to integers
los_data['vdate_year'] = los_data['vdate_year'].astype(float)
los_data['vdate_month'] = los_data['vdate_month'].astype(float)
los_data['vdate_day'] = los_data['vdate_day'].astype(float)
los_data['vdate_day_of_week'] = los_data['vdate_day_of_week'].astype(float)

In [ ]:
# Feature Engineer discharged date

# Extract vdate individual elements
los_data['discharged_year'] = los_data['discharged'].dt.year
los_data['discharged_month'] = los_data['discharged'].dt.month
los_data['discharged_day'] = los_data['discharged'].dt.day
los_data['discharged_day_of_week'] = los_data['discharged'].dt.dayofweek

# Convert vdate elements to integers
los_data['discharged_year'] = los_data['discharged_year'].astype(float)
los_data['discharged_month'] = los_data['discharged_month'].astype(float)
los_data['discharged_day'] = los_data['discharged_day'].astype(float)
los_data['discharged_day_of_week'] = los_data['discharged_day_of_week'].astype(float)

In [ ]:
# Drop original vdate and discharge fields
column_to_drop = ['vdate', 'discharged']
los_data = los_data.drop(column_to_drop, axis=1)

In [ ]:
# Create heatmap of multicollinearity

In [ ]:
corr_matrix = los_data[['hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi', 'pulse','respiration',
                       'vdate_year', 'vdate_month', 'vdate_day', 'vdate_day_of_week', 
                        'discharged_year', 'discharged_month', 'discharged_day', 'discharged_day_of_week' ]].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Multicollinearity Heatmap')
plt.show()

In [ ]:
# vdate and discharge date are too correlated
# will not include vdate in model
# Drop unneeded vdate

column_to_drop = ['vdate_year', 'vdate_month', 'vdate_day', 'vdate_day_of_week']
los_data = los_data.drop(column_to_drop, axis=1)

In [ ]:
# Rerun correlation matrix
corr_matrix = los_data[['hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro', 'creatinine', 'bmi', 'pulse','respiration', 
                        'discharged_year', 'discharged_month', 'discharged_day', 'discharged_day_of_week']].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Multicollinearity Heatmap')
plt.show()

In [ ]:
los_data['longlengthofstay']=los_data['longlengthofstay'].astype('int')
los_data['facid_A']=los_data['facid_A'].astype('int')
los_data['facid_B']=los_data['facid_B'].astype('int')
los_data['facid_C']=los_data['facid_C'].astype('int')
los_data['facid_D']=los_data['facid_D'].astype('int')
los_data['facid_E']=los_data['facid_E'].astype('int')

los_data.info()

In [ ]:
## Ended up dropping discharged date too as was giving errors in model
column_to_drop = ['discharged_year', 'discharged_month', 'discharged_day', 'discharged_day_of_week']
los_data = los_data.drop(column_to_drop, axis=1)

In [ ]:
# Set up scaling and encoding transformers
# los_data.head()

In [ ]:
# Get all the feature related names and indices for later use

feature_names = los_data.columns.tolist()
feature_names.remove('longlengthofstay')

target_names = ['Extended LOS']

num_col_names = ['rcount', 'hematocrit', 'neutrophils', 'sodium', 'glucose', 'bloodureanitro', 'creatinine', 
                    'bmi', 'pulse', 'respiration']
num_col_indice = [feature_names.index(nc) for nc in num_col_names] # need this to fit alibi needs

cat_col_names = [fn for fn in feature_names if fn not in num_col_names]
cat_col_indice = [feature_names.index(cc) for cc in cat_col_names]

In [ ]:
# Create the data scaling preprocessors 

scaler = MinMaxScaler() #StandardScaler()

# only numerical columns are scaled here
# in the future, categorical columns will need encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, num_col_indice)
    ],
    remainder='passthrough',
    sparse_threshold=0
)

In [ ]:
# Split data into training and test

In [ ]:
# Split the dataset into features (X) and the target variable (y)

X = los_data[feature_names] 
y = los_data['longlengthofstay']

# Split the data into a training set and a testing
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2)

#
#
# Turn these into numpy arrays because alibi works with numpy array only
#
#
X_train, X_test, y_train, y_test = X_train_df.to_numpy(), X_test_df.to_numpy(), y_train_df.to_numpy(), y_test_df.to_numpy()

# Fit preprocessing transformations to the entire data set
# This is important especially when categorical data is involved
preprocessor.fit(X.to_numpy())

# Transform both train and test data
X_train_tf = preprocessor.transform(X_train)
X_test_tf = preprocessor.transform(X_test)

#*************************************************
# In summary, the following are finalized data
#    - DataFrame's
#                X, y 
#                X_train_df, X_test_df, y_train_df, y_test_df
#    - numpy array's
#                X_train, X_test, y_train, y_test
#    - normalized/encoded numpy array's
#                X_train_tf, X_test_tf
#**************************************************

In [ ]:
## RUN SUPERVISED LEARNING MODELS
##
## ***NOTE: From here on all train and test data are in numpy format (no longer DataFrame)
##

In [ ]:
# Logistic Regression

In [ ]:
# Create a logistic regression model
logistic_regression = LogisticRegression(max_iter=5000)

# Fit the model to the training data
logistic_regression.fit(X_train_tf, y_train)

# Make predictions on the test set
y_pred_LR = logistic_regression.predict(X_test_tf)

# Probability of long stay
y_proba_LR = logistic_regression.predict_proba(X_test_tf)[:, 1]

In [ ]:
# Calculate model statistics

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred_LR)
report = classification_report(y_test, y_pred_LR)
conf_matrix = confusion_matrix(y_test, y_pred_LR)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred_LR)
recall = recall_score(y_test, y_pred_LR)
f1 = f1_score(y_test, y_pred_LR)

# Calculate AUC (Area Under the ROC Curve)
auc_LR = roc_auc_score(y_test, y_proba_LR)

# Get coefficients and feature names
coefficients = logistic_regression.coef_

In [ ]:
# Print the model statististics - Logistic Regression

In [ ]:
print(f'Accuracy: {accuracy}')
print('\nClassification Report:\n', report)
print('Confusion Matrix:\n', conf_matrix)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"AUC: {auc_LR:.2f}")

# Print the model's coefficients and intercept
print("Features:", feature_names)
print("Coefficients:", logistic_regression.coef_)
print("Intercept:", logistic_regression.intercept_)


In [ ]:
# Since the dataset is highly imbalanced, data balancing should be added in the future
# Furthermore, the area of precision recall curve (AUPRC) is a better indicator of 
# prediction performance. In sklearn, this value is estimated by the following function.

from sklearn.metrics import average_precision_score
auprc = average_precision_score(y_test, y_proba_LR)
print(f"AUPRC: {auprc}")

In [ ]:
# Gradiant Boosting Classifier Model

In [ ]:
# Create a Gradient Boosting Classifier
gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, max_depth=3, min_samples_leaf=2, min_samples_split=10)

# Fit the model to the training data (Note: the data are in numpy format, which is needed by XAI packages)
gradient_boosting_classifier.fit(X_train_tf, y_train)

# Make predictions on the test data
y_pred_GB = gradient_boosting_classifier.predict(X_test_tf)

# Probability of long stay
y_proba_GB = gradient_boosting_classifier.predict_proba(X_test_tf)[:, 1]


In [ ]:
#************
# Because DiCE library doesn't accept numpy, so fit another GB model here using DataFrame training data
#************
gbc_df = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, max_depth=3, min_samples_leaf=2, min_samples_split=10)

# Fit the model to the training data using DataFrame data, and no standardization/encoding is needed
# since the tree models don't really need them
gbc_df.fit(X_train_df, y_train_df)

In [ ]:
# Calculate model statistics

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred_GB)
report = classification_report(y_test, y_pred_GB)
conf_matrix = confusion_matrix(y_test, y_pred_GB)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred_GB)
recall = recall_score(y_test, y_pred_GB)
f1 = f1_score(y_test, y_pred_GB)

# Calculate AUC (Area Under the ROC Curve)
auc_GB = roc_auc_score(y_test, y_proba_GB)

In [ ]:
print(f'Accuracy: {accuracy}')
print('\nClassification Report:\n', report)
print('Confusion Matrix:\n', conf_matrix)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"AUC: {auc_GB:.2f}")
#print("Feature Importance:", feature_importance)

# Print the model's coefficients and intercept
print("Features: ", feature_names)
print("Feature Importance: ", gradient_boosting_classifier.feature_importances_)

In [ ]:
auprc = average_precision_score(y_test, y_proba_GB)
print(f"AUPRC: {auprc}")

In [ ]:
# The gbc_df model should perform similarily. Let's do a sanity check here.
print(f"AUPRC for gbc_df model: {average_precision_score(y_test_df, gbc_df.predict_proba(X_test_df)[:,1])}")

# Explainable AI Methods

In [ ]:
### XAI libraries - SHAP and InterpretML (with SHAP)
import shap
from interpret import show
from interpret.blackbox import ShapKernel
from interpret.blackbox import LimeTabular
from interpret.blackbox import PartialDependence
#gradient_boosting_classifier.predict_proba([X_test.iloc[0]])


In [ ]:
# Local explanation using Kernel SHAP using interpret
predict_fn = lambda x: gradient_boosting_classifier.predict_proba(preprocessor.transform(x))[:,1]
gb_shap = ShapKernel(predict_fn, shap.sample(X_train, 100), feature_names=feature_names)
gb_shap_local = gb_shap.explain_local(X_test[:100], y_test[:100])
show(gb_shap_local)

In [ ]:
# Local explanation with LIME using interpret
gb_lime = LimeTabular(predict_fn, X_train, feature_names=feature_names)
gb_lime_local = gb_lime.explain_local(X_test[0:100], y_test[0:100])
show(gb_lime_local)


In [ ]:
# Global explanation with PDP using interpret
predict_fn = lambda x: gradient_boosting_classifier.predict_proba(preprocessor.transform(x))[:, 1]
gb_pdp = PartialDependence(predict_fn, X_train, feature_names=feature_names)
gb_pdp_global = gb_pdp.explain_global()
show(gb_pdp_global)

In [ ]:
# Explanation using the SHAP library directly
# The first parameter must be the model itself. We can't use the trick of predict_fn here.
# Fortunately, for tree models, we shouldn't have used data normalization anyway.

explainer = shap.TreeExplainer(gradient_boosting_classifier, feature_names=feature_names)
# Calculate shapley values for test data
#shap_values = explainer.shap_values(X_test)
shap_values = explainer(X_test_tf[0:100])

# Investigating the values (classification problem)
# class 0 = contribution to class 1
# class 1 = contribution to class 2
#print(shap_values[1].shape)
#shap_values

explainer.expected_value

In [ ]:
# SHAP library visualization

shap.initjs()

# Local explanation using Force plot

shap.plots.force(shap_values[33])

In [ ]:
# Local explanation using SHAP's Water Fall plot 

shap.plots.waterfall(shap_values[33])

In [ ]:
# Global explanation using SHAP Feature Summary

shap.plots.beeswarm(shap_values)


In [ ]:
# XAI library for counterfactual analysis using DiCE

import dice_ml

data_dice = dice_ml.Data(dataframe=pd.concat([X_train_df, y_train_df], axis=1), 
                         continuous_features=num_col_names,
                         outcome_name='longlengthofstay')
model_dice = dice_ml.Model(model=gbc_df,
                           backend='sklearn')
exp_dice = dice_ml.Dice(data_dice, model_dice, method='random')

cf = exp_dice.generate_counterfactuals(X_test_df[0:1], total_CFs=10, desired_class='opposite')

In [ ]:
# Displaying counterfactuals as a dataframe

cf.visualize_as_dataframe(show_only_changes=True)

In [ ]:
# PDP, ICE, ALE and many other methods using the alibi XAI library

# PDP using alibi

import alibi.explainers as aliexp
predict_fn = lambda x: gradient_boosting_classifier.predict_proba(preprocessor.transform(x))[:, 1]
pd_explainer = aliexp.PartialDependence(predictor=predict_fn,
                              feature_names=feature_names,
                                target_names=target_names,
                                categorical_names=cat_col_names)
pd_exp = pd_explainer.explain(X=X_test,
                        kind='average')

In [ ]:
%xmode minimal

fig, axs = plt.subplots(3, 2, figsize=(7.5, 10), sharey='none', layout='constrained')
aliexp.plot_pd(exp=pd_exp, ax=axs, features=[0, 4, 5, 12, 14, 15])

In [ ]:
# ALE plot using alibi

from alibi.explainers import ALE
from alibi.explainers.ale import plot_ale

predict_fn = lambda x: gradient_boosting_classifier.predict_proba(preprocessor.transform(x))[:,1]
ale = ALE(predict_fn, feature_names=feature_names)
exp = ale.explain(X_test)


In [ ]:

plot_ale(exp, n_cols=2, sharey=None, features=[0, 4, 5, 12, 14, 15], line_kw={'label': 'Probability of "longlengthofstay" class'}, fig_kw={'figheight': 10, 'figwidth': 7.5})


In [ ]:
# Data explanation and visualization using SweetViz
#%pip install sweetviz

import sweetviz
los_data_viz = sweetviz.analyze([los_data, 'LOS'], target_feat='longlengthofstay')
los_data_viz.show_html('sweetviz-los.html')